In [2]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score

In [3]:
# Your selected columns
selected_num_columns = ['LotFrontage', 'LotArea', 'TotalBsmtSF', 'GrLivArea', 'SalePrice']
categorical_columns_filtered = ['Street', 'LandContour', 'LandSlope', 'Utilities', 'Neighborhood',
                                  'Condition1', 'Condition2', 'HouseStyle', 'BldgType', 'OverallQual',
                                  'OverallCond', 'RoofStyle', 'Exterior1st', 'ExterCond', 'BsmtCond',
                                  'BsmtFinType1', 'CentralAir', 'Heating', 'KitchenQual',
                                  'TotRmsAbvGrd', 'GarageType', 'GarageCond', 'PavedDrive',
                                  'SaleType', 'SaleCondition', 'Fireplaces', 'GarageCars']

In [4]:
train_set = pd.read_csv('../data/train.csv')
test_set = pd.read_csv('../data/test.csv')

In [8]:
# Define the target and features
# SalePrice is the target, so exclude it from features.
X = train_set[[col for col in selected_num_columns if col != 'SalePrice'] + categorical_columns_filtered]
y = train_set['SalePrice']

In [9]:





# Preprocessing for numerical features: imputation and scaling
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical features: imputation and one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine the numeric and categorical transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, [col for col in selected_num_columns if col != 'SalePrice']),
        ('cat', categorical_transformer, categorical_columns_filtered)
    ])

# Define base models for stacking
base_models = [
    ('ridge', Ridge(alpha=1.0)),
    ('lasso', Lasso(alpha=0.1)),
    ('gbr', GradientBoostingRegressor(n_estimators=100, random_state=42))
]

# Define the final estimator (meta-model)
final_estimator = LinearRegression()

# Create the stacking regressor
stacking_reg = StackingRegressor(
    estimators=base_models,
    final_estimator=final_estimator,
    cv=5  # using 5-fold cross validation for stacking
)

# Build a pipeline that first preprocesses the data then fits the stacking regressor
stacking_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('stacking', stacking_reg)
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the stacking model
stacking_pipeline.fit(X_train, y_train)

# Evaluate the model
score = stacking_pipeline.score(X_test, y_test)
print("Stacking model R^2 score on test data: {:.3f}".format(score))


c:\Users\maryi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 66719729951.07739, tolerance: 696659484.3571945
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\maryi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 47916292135.72864, tolerance: 537415025.1745833
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\maryi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 112590348325.74149, tolerance: 572016182.2224233
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\maryi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:

Stacking model R^2 score on test data: 0.893
